In [2]:
from MatlabFuncs import *
from model import *
from triangle import *
from bamg import bamg
from savevars import *
import plotdoc
from loadmodel import *
import os
from os.path import exists

from export_netCDF import export_netCDF

from scipy.io import loadmat
from m1qn3inversion import *
import numpy as np
from ContourToNodes import *
from solve import *

#import lhsmdu #install in terminal 'pip install lhsmdu'
import matplotlib.pyplot as plt
import pickle

import numpy as np
from osgeo import gdal
from matplotlib import pyplot as plt

from InterpFromGridToMesh import InterpFromGridToMesh
from InterpFromMeshToMesh2d import InterpFromMeshToMesh2d

import copy
import utilities
from reinitializelevelset import *

In [ ]:
region = 'SAtoES'
start_year  = 1985.

In [ ]:
# Load a separate model to extract the friction coefficient and rheology
md_friction = loadmodel("./Models/SAtoES_friction_coefficient.nc")
friction_coefficient = md_friction.results.StressbalanceSolution.FrictionCoefficient
rheology_B = md_friction.materials.rheology_B
md_friction = None

# Load the original model
md = loadmodel("./Models/SAtoES_inversion.nc")
md.friction.coefficient = friction_coefficient
md.materials.rheology_B = rheology_B

md.levelset.spclevelset = np.nan * np.ones((md.mesh.numberofvertices))
pos = md.mesh.vertexonboundary == 1
#pos = np.where(md.mesh.vertexonboundary)
md.levelset.spclevelset[pos] = md.mask.ice_levelset[pos]
md.levelset.migration_max = 1e10
plotmodel(md, 'data', md.mask.ice_levelset)

In [ ]:
## Relaxation
relaxation_years = 30 #try 5 years if 1 runs quickly

# Special post-processing of inverted friction coefficient
filename = 'Exp/' + region + '_coeffront_after_inversion.exp'
if os.path.isfile(filename):
  print('adjusting inverted friction coefficient at the glacier fronts')
  pos = find(ContourToNodes(md.mesh.x, md.mesh.y, filename, 1))
  md.friction.coefficient[pos] = 10

md.initialization.pressure = np.zeros([md.mesh.numberofvertices,1])
md.initialization.temperature = 250*np.ones([md.mesh.numberofvertices,1]) #temperature is in kelvin

# Set parameters
md.inversion.iscontrol=0;
md.timestepping.start_time = start_year;
md.timestepping.time_step  = .02;
md.timestepping.final_time = start_year + relaxation_years;
md.settings.output_frequency = (1/md.timestepping.time_step)/5; # 5/yr

In [ ]:
# define fjord walls/flush where bed > 0 (code added 8/3/22)
n_buffer = 2
for i in range(n_buffer):
    elements = md.mesh.elements.astype(int)-1
    nodes_edge = elements[np.where(np.sum(md.mask.ice_levelset[elements] == -1, axis=1) == 2)[0]]
    nodes_edge = np.unique(nodes_edge.ravel())
    nodes_bed = np.where(md.geometry.bed > 0)[0]
    nodes_edge_bed = np.array(list(set(nodes_edge) & set(nodes_bed)))
    plt.plot(md.mesh.x[nodes_edge_bed], md.mesh.y[nodes_edge_bed], 'r.', markersize=10)

    md.mask.ice_levelset[nodes_edge_bed] = -1
    md.geometry.thickness[nodes_edge_bed] = 10
    md.geometry.surface[nodes_edge_bed] = md.geometry.bed[nodes_edge_bed] + md.geometry.thickness[nodes_edge_bed]

    md.friction.coefficient[nodes_edge_bed] = 200

In [ ]:
# defining thermal and sub. discharge at every ISSM mesh node 

# load raster of basins
ds = gdal.Open('./Cheat_matfiles/tidewaterbasins_rignotid.mat_tidewaterbasins.tif', gdal.GA_ReadOnly)
rb = ds.GetRasterBand(1)
basins_array = rb.ReadAsArray()

gt = ds.GetGeoTransform()
ulx, xres, xskew, uly, yskew, yres  = gt
lrx = ulx + (ds.RasterXSize * xres)
lry = uly + (ds.RasterYSize * yres)
x = np.arange(ulx, lrx,  xres)
y = np.arange(lry, uly, -yres)

In [ ]:
# interpolate basin to each mesh node 
px = np.array( ((md.mesh.x - gt[0]) / gt[1]).astype(int) )
py = np.array( ((md.mesh.y - gt[3]) / gt[5]).astype(int) )

basins_mesh = basins_array[py.astype(int), px.astype(int)]

# ISSM wants basins to be numbered from 1 to 4 with the basins that we don't care about numbered 0
basins_mesh[basins_mesh==13] = 1
basins_mesh[basins_mesh==52] = 2
basins_mesh[basins_mesh==53] = 3
basins_mesh[basins_mesh==90] = 4
basins_mesh[(basins_mesh<1) | (basins_mesh>4)] = 0

# find basin id for each element
basins_elements = basins_mesh[md.mesh.elements-1]
basins_elements = np.max(basins_elements, axis=1)

plotmodel(md, 'data', basins_elements)

In [ ]:
# Assigning ocean thermal (EN4) and subglacial discharge (RACMO2.3p2) forcings
from scipy_io_utils import *

# Load historical forcing data
glacier_list = list()
m = loadmat('./Cheat_matfiles/glacier0013.mat')
glacier0013 = m['glacier0013']
m = loadmat('./Cheat_matfiles/glacier0052.mat')
glacier0052 = m['glacier0052']
m = loadmat('./Cheat_matfiles/glacier0053.mat')
glacier0053 = m['glacier0053']
m = loadmat('./Cheat_matfiles/glacier0090.mat')
glacier0090 = m['glacier0090']

# Thermal forcing
t = glacier0013['EN4']['t']
TF = glacier0013['EN4']['TF']

# Discharge
t = glacier0013['RACMO']['Q']
Q = glacier0013['RACMO']['Q']

from frontalforcingsrignot import frontalforcingsrignot
md.frontalforcings = frontalforcingsrignot()
valid = ~np.isnan(glacier0013['EN4']['TF'])
md.frontalforcings.thermalforcing = np.zeros( (md.mesh.numberofvertices+1, len(glacier0013['EN4']['TF'][valid])) )
md.frontalforcings.subglacial_discharge = np.zeros( (md.mesh.numberofvertices+1, len(glacier0013['RACMO']['Q'])) )

In [ ]:
# Assign forcings to correct node 

# step 1: find which nodes belong to each basin using basin_mesh
get_indexes = lambda basins_mesh, xs: [i for (y, i) in zip(xs, range(len(xs))) if basins_mesh == y]
glacier13_nodes = get_indexes(1,basins_mesh)
glacier52_nodes = get_indexes(2,basins_mesh)
glacier53_nodes = get_indexes(3,basins_mesh)
glacier90_nodes = get_indexes(4,basins_mesh)

# NOTE: Denis combined steps 2 and 3: we select the rows and set to the TF time series
# step 2: select() rows from frontalforcings.thermal that correspond to step 1 nodes
md.frontalforcings.thermalforcing[glacier13_nodes,:] = glacier0013['EN4']['TF'][valid]
md.frontalforcings.thermalforcing[glacier52_nodes,:] = glacier0052['EN4']['TF'][valid]
md.frontalforcings.thermalforcing[glacier53_nodes,:] = glacier0053['EN4']['TF'][valid]
md.frontalforcings.thermalforcing[glacier90_nodes,:] = glacier0090['EN4']['TF'][valid]

# step 2: select() rows from frontalforcings.subglacial_discharge that correspond to step 1 nodes
md.frontalforcings.subglacial_discharge[glacier13_nodes,:] = glacier0013['RACMO']['Q'] * 86400
md.frontalforcings.subglacial_discharge[glacier52_nodes,:] = glacier0052['RACMO']['Q'] * 86400
md.frontalforcings.subglacial_discharge[glacier53_nodes,:] = glacier0053['RACMO']['Q'] * 86400
md.frontalforcings.subglacial_discharge[glacier90_nodes,:] = glacier0090['RACMO']['Q'] * 86400

# NOTE: And now, set the last row to the time
md.frontalforcings.thermalforcing[-1,:] = glacier0013['EN4']['t'][valid]
md.frontalforcings.subglacial_discharge[-1,:] = glacier0013['RACMO']['t']

# NOTE: All times are the same for the TF time series for each glacier so we just set the times equal to the glacier0013 times


## Load the parameters for the ensemble

In [ ]:
import pickle
with open('ens_dict.pickle', 'rb') as f:
    ens_data = pickle.load(f)

## Loop through ensemble members and launch ISSM on the cluster

In [ ]:
for trial_name, parameters in ens_data.items():
    print('Launching ' + trial_name)
    
    # We set the transient parameters
    md.transient.ismovingfront=1
    md.transient.isthermal=0
    md.transient.isstressbalance=1
    md.transient.ismasstransport=1
    md.transient.isgroundingline=1
    md.groundingline.migration = 'SubelementMigration'

    # We set the ocean conditions
    md.frontalforcings.basin_id = basins_elements
    md.frontalforcings.num_basins = 4

    # We set the calving model
    from calvingvonmises import calvingvonmises
    md.calving = calvingvonmises()

    # Set the requested outputs
    md.stressbalance.requested_outputs=['default']
    md.transient.requested_outputs=['default','IceVolumeAboveFloatation','BasalforcingsGroundediceMeltingRate','CalvingMeltingrate']

    # SMCE cluster
    from eis_nasa_smce import eis_nasa_smce
    md.cluster = eis_nasa_smce()
    md.cluster.name = 'pcluster.sealevel.eis.smce.nasa.gov'
    md.cluster.partition = 'sealevel-c5n18xl-demand'
    md.cluster.login = 'mpascual'
    md.cluster.idfile = '~/.ssh/id_rsa'
    md.cluster.executionpath = '/efs/mpascual/issm_execution_cluster'
    md.cluster.cpuspernode = 14

    md.settings.waitonlock = 0

    # Go solve
    md.miscellaneous.name = region + '_' + trial_name
    md.verbose.solution=1
    from solve import solve
    md = solve(md, 'sb')

    # Save this model 
    export_netCDF(md, '/efs/mpascual/GrIS_Outlet_Glacier_Seasonal_dhdt/Models/' + \
                  region + '_' + '_hindcast_EN4_RACMO_' + trial_name + '_sent2cluster.nc')
    
    print('')
    break

In [ ]:
md = loadmodel("/efs/mpascual/GrIS_Outlet_Glacier_Seasonal_dhdt/Models/SAtoES__hindcast_EN4_RACMO_Trial0_sent2cluster.nc")
md = loadresultsfromcluster(md)
#load md cluster results check sb fric coeff field
fig = plt.figure(figsize=(15,15))
plotmodel(md, 'data', md.results.StressbalanceSolution.Vel[:,0], \
          'mask', md.mask.ice_levelset<0, 'colormap', 'jet', 'caxis', [0,1500])

In [ ]:
# Global setup of the base model

###################################################
## 1) Load the _param model + common parameters  ##
###################################################
md = loadmodel('/efs/GrIS_Outlet_Glacier_Seasonal_dhdt/Models/SAtoES_Param.nc')

#Setup
region = 'SAtoES'
start_year  = 1985.

# Mesh sizing
triangleresolution = 1000;
hmin = 300;
hmax = 10000;

# Mesh
md = model()
md = triangle(md,'./Exp/' +region+ '.exp',triangleresolution) # set up mesh

md = bamg(md,'hmin',hmin,'hmax',hmax,'field',vel,'err',2,'hmaxVertices',hmaxVertices);


###################################
## 2) for loop for each ensemble ##
###################################
import pickle
with open('ens_dict.pickle', 'rb') as f:
    ens_data = pickle.load(f)
    
for trial_name, parameters in ens_data.items():
    print('Launching ' + trial_name)
    
    ####################################################
    ## set friction coefficient and rheology with LHS ##
    ####################################################
    #set sliding law 
    md.friction.p = LHS value #(round to nearest whole number?)
    
    #set ice rheology
    from cuffey import cuffey 
    #Vary Celsius range with LHS
    temperature = -10 + (-10*LHS value) #(-15 = cold, -1 = warm)
    # convert celsius to kelvin
    temperature = temperature + 273.15 
    #Calculate rigidity
    rigidity = cuffey(temperature)
    #Set rigidity calculation to rheology
    md.materials.rheology_B = rigidity
    
    ####################
    ## run inversion ##
    ####################
    md.inversion.iscontrol = 1 
    from solve import solve
    md = solve(md, 'sb')
    
    # Save this model 
    export_netCDF(md, '/efs/GrIS_Outlet_Glacier_Seasonal_dhdt/Models/inversion_models/' + \
                  region + '_' + '_hindcast_EN4_RACMO_inversion' + trial_name + '_sent2cluster.nc')
    
    ###############################################
    ## extract friction coefficient and rheology ##
    ###############################################
    friction_coefficient = md.results.StressbalanceSolution.FrictionCoefficient
    rheology_B = md.materials.rheology_B
    #md_friction = None
    
    # Load the original model
    #md = loadmodel("./Models/SAtoES_inversion.nc")
    md.friction.coefficient = friction_coefficient
    md.materials.rheology_B = rheology_B

    #################################################
    ## Interpolate the 1985 DEM surface elevations ##
    #################################################
    z_1985dem = utilities.interp1985DEM(md.mesh.x, md.mesh.y)
    # Select the points that will be used to create the linear relationship between the two surfaces
    pos = (~np.isnan(z_1985dem - md.geometry.surface)) & (z_1985dem > 700) & (z_1985dem < 1000)
    # Find the linear relationship
    p = np.polyfit(md.geometry.surface[pos], z_1985dem[pos], 1)
    z_polyval = np.polyval(p, md.geometry.surface[pos])
    residuals = z_1985dem[pos] - z_polyval
    # Shift the surface
    surf_shifted = utilities.surface_shift(md, md.geometry.surface, z_1985dem)
    md.geometry.surface = surf_shifted
    
    # Find the points where bed < 0 and the new surface > 42 m
    ice_levelset_original = copy.deepcopy(md.mask.ice_levelset)
    pos = (md.geometry.bed < 0) & (md.geometry.surface > 42) & (md.mask.ice_levelset > 0) & (md.mesh.vertexonboundary == 0)

    # Fill in these points with ice
    md.mask.ice_levelset[pos] = -1
    md.mask.ice_levelset = reinitializelevelset(md, md.mask.ice_levelset)

    # Extrapolate friction coefficient
    from scipy.interpolate import griddata
    valid = (ice_levelset_original < 0) & (md.geometry.surface < 300)
    extrap = (md.geometry.bed < 0) & (md.geometry.surface > 42) & (ice_levelset_original > 0) & (md.mesh.vertexonboundary == 0)
    friction_coefficient_extrap = griddata((md.mesh.x[valid], md.mesh.y[valid]), \
                                            md.friction.coefficient[valid], \
                                           (md.mesh.x[extrap], md.mesh.y[extrap]), \
                                            method='nearest')
    md.friction.coefficient[extrap] = friction_coefficient_extrap


    md.levelset.spclevelset = np.nan * np.ones((md.mesh.numberofvertices))
    pos = md.mesh.vertexonboundary == 1
    md.levelset.spclevelset[pos] = md.mask.ice_levelset[pos]
    md.levelset.migration_max = 1e10
    
    ####################
    ## set relaxation ##
    ####################
    relaxation_years = 30 #try 5 years if 1 runs quickly
    
    # Special post-processing of inverted friction coefficient
    filename = 'Exp/' + region + '_coeffront_after_inversion.exp'
    if os.path.isfile(filename):
        print('adjusting inverted friction coefficient at the glacier fronts')
        pos = find(ContourToNodes(md.mesh.x, md.mesh.y, filename, 1))
        md.friction.coefficient[pos] = 10
        
    md.initialization.pressure = np.zeros([md.mesh.numberofvertices,1])
    md.initialization.temperature = 250*np.ones([md.mesh.numberofvertices,1]) #temperature is in kelvin
        
    # Set parameters
    md.inversion.iscontrol=0;
    md.timestepping.start_time = start_year;
    md.timestepping.time_step  = .02;
    md.timestepping.final_time = start_year + relaxation_years;
    md.settings.output_frequency = (1/md.timestepping.time_step)/5; # 5/yr
    
    #############################
    ## input seasonal forcings ##
    #############################
        
    # defining thermal and sub. discharge at every ISSM mesh node 
    # load raster of basins
    ds = gdal.Open('./Cheat_matfiles/tidewaterbasins_rignotid.mat_tidewaterbasins.tif', gdal.GA_ReadOnly)
    rb = ds.GetRasterBand(1)
    basins_array = rb.ReadAsArray()
        
    gt = ds.GetGeoTransform()
    ulx, xres, xskew, uly, yskew, yres  = gt
    lrx = ulx + (ds.RasterXSize * xres)
    lry = uly + (ds.RasterYSize * yres)
    x = np.arange(ulx, lrx,  xres)
    y = np.arange(lry, uly, -yres)
        
    # interpolate basin to each mesh node 
    px = np.array( ((md.mesh.x - gt[0]) / gt[1]).astype(int) )
    py = np.array( ((md.mesh.y - gt[3]) / gt[5]).astype(int) )
    basins_mesh = basins_array[py.astype(int), px.astype(int)]
        
    # ISSM wants basins to be numbered from 1 to 4 with the basins that we don't care about numbered 0
    basins_mesh[basins_mesh==13] = 1
    basins_mesh[basins_mesh==52] = 2
    basins_mesh[basins_mesh==53] = 3
    basins_mesh[basins_mesh==90] = 4
    basins_mesh[(basins_mesh<1) | (basins_mesh>4)] = 0

    # find basin id for each element
    basins_elements = basins_mesh[md.mesh.elements-1]
    basins_elements = np.max(basins_elements, axis=1)
        
    # Assigning ocean thermal (EN4) and subglacial discharge (RACMO2.3p2) forcings
    from scipy_io_utils import *

    # Load historical forcing data
    glacier_list = list()
    m = loadmat('./Cheat_matfiles/glacier0013.mat')
    glacier0013 = m['glacier0013']
    m = loadmat('./Cheat_matfiles/glacier0052.mat')
    glacier0052 = m['glacier0052']
    m = loadmat('./Cheat_matfiles/glacier0053.mat')
    glacier0053 = m['glacier0053']
    m = loadmat('./Cheat_matfiles/glacier0090.mat')
    glacier0090 = m['glacier0090']

    # Thermal forcing
    t = glacier0013['EN4']['t']
    TF = glacier0013['EN4']['TF']

    # Discharge
    t = glacier0013['RACMO']['Q']
    Q = glacier0013['RACMO']['Q']

    from frontalforcingsrignot import frontalforcingsrignot
    md.frontalforcings = frontalforcingsrignot()
    valid = ~np.isnan(glacier0013['EN4']['TF'])
    md.frontalforcings.thermalforcing = np.zeros( (md.mesh.numberofvertices+1, len(glacier0013['EN4']['TF'][valid])) )
    md.frontalforcings.subglacial_discharge = np.zeros( (md.mesh.numberofvertices+1, len(glacier0013['RACMO']['Q'])) )
        
    # Assign forcings to correct node 
    # step 1: find which nodes belong to each basin using basin_mesh
    get_indexes = lambda basins_mesh, xs: [i for (y, i) in zip(xs, range(len(xs))) if basins_mesh == y]
    glacier13_nodes = get_indexes(1,basins_mesh)
    glacier52_nodes = get_indexes(2,basins_mesh)
    glacier53_nodes = get_indexes(3,basins_mesh)
    glacier90_nodes = get_indexes(4,basins_mesh)

    # NOTE: Denis combined steps 2 and 3: we select the rows and set to the TF time series
    # step 2: select() rows from frontalforcings.thermal that correspond to step 1 nodes
    md.frontalforcings.thermalforcing[glacier13_nodes,:] = glacier0013['EN4']['TF'][valid]
    md.frontalforcings.thermalforcing[glacier52_nodes,:] = glacier0052['EN4']['TF'][valid]
    md.frontalforcings.thermalforcing[glacier53_nodes,:] = glacier0053['EN4']['TF'][valid]
    md.frontalforcings.thermalforcing[glacier90_nodes,:] = glacier0090['EN4']['TF'][valid]

    # step 2: select() rows from frontalforcings.subglacial_discharge that correspond to step 1 nodes
    md.frontalforcings.subglacial_discharge[glacier13_nodes,:] = glacier0013['RACMO']['Q'] * 86400
    md.frontalforcings.subglacial_discharge[glacier52_nodes,:] = glacier0052['RACMO']['Q'] * 86400
    md.frontalforcings.subglacial_discharge[glacier53_nodes,:] = glacier0053['RACMO']['Q'] * 86400
    md.frontalforcings.subglacial_discharge[glacier90_nodes,:] = glacier0090['RACMO']['Q'] * 86400

    # NOTE: And now, set the last row to the time
    md.frontalforcings.thermalforcing[-1,:] = glacier0013['EN4']['t'][valid]
    md.frontalforcings.subglacial_discharge[-1,:] = glacier0013['RACMO']['t']
    
    #####################
    ## set Calving law ##
    #####################
    from calvingvonmises import calvingvonmises
    md.calving = calvingvonmises()
    md.calving.stress_threshold_groundedice = md.calving.stress_threshold_groundedice + (md.calving.stress_threshold_groundedice * LHS value)
    
    ###################################
    ## set the transient parameters ##
    ###################################
    md.transient.ismovingfront=1
    md.transient.isthermal=0
    md.transient.issmb = 0
    md.thermal.isenthalpy = 0
    md.transient.isstressbalance=1
    md.transient.ismasstransport=1
    md.transient.isgroundingline=1
    md.groundingline.migration = 'SubelementMigration'
    
    # We set the ocean conditions
    md.frontalforcings.basin_id = basins_elements
    md.frontalforcings.num_basins = 4
    
    # SMCE pCluster
    from eis_nasa_smce import eis_nasa_smce
    md.cluster = eis_nasa_smce()
    md.cluster.name = 'pcluster.sealevel.eis.smce.nasa.gov'
    md.cluster.partition = 'sealevel-c5n18xl-demand'
    md.cluster.login = 'mpascual'
    md.cluster.idfile = '~/.ssh/id_rsa'
    md.cluster.executionpath = '/efs/mpascual/issm_execution_cluster'
    md.cluster.cpuspernode = 36
    
    md.settings.waitonlock = 0
    
    # Go solve
    md.miscellaneous.name = region + '_' + trial_name
    md.verbose.solution=1
    from solve import solve
    md = solve(md, 'tr')

    # Save this model 
    export_netCDF(md, '/efs/dfelikso/GrIS_Outlet_Glacier_Seasonal_dhdt/Models/transient_models/' + \
                  region + '_' + '_hindcast_EN4_RACMO_transient' + trial_name + '_sent2cluster.nc')
    
    print('')